In [20]:
import cv2
import pytesseract
import easyocr
import numpy as np
import re

# Image Preprocessing: Adjust contrast, denoise, sharpen, and adaptive thresholding
def adjust_contrast(image):
    return cv2.equalizeHist(image)

def denoise_image(image):
    return cv2.fastNlMeansDenoising(image, None, 30, 7, 21)

def sharpen_image(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def adaptive_threshold(image):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Deskewing: Align the image if it is skewed
def deskew_image(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Multiple passes of OCR with different preprocessing techniques
def multiple_pass_ocr(image):
    processed_images = [sharpen_image(image), adaptive_threshold(image), denoise_image(image)]
    all_extracted_text = []

    for img in processed_images:
        extracted_text = pytesseract.image_to_string(img, config=r'--oem 3 --psm 6')
        all_extracted_text.append(extracted_text)

    return ' '.join(all_extracted_text)  # Combine all extracted texts

# Regex to extract specific details like brand, pack size, product name, etc.
def extract_details_using_regex(text):
    # Enhanced regex to match both oz and g, handling different formats e.g. "10 OZ" and "283.5 g"
    pack_size_matches = re.findall(r'(\d+\.?\d*)\s*(oz|g|kg|ml|L)', text, re.IGNORECASE)

    # Flexible regex for brand name - captures common brand names like "Lays", "BRAN", etc.
    brand_name_match = re.search(r'\b(Lays|BRAN|Brand|Potato Chips)\b', text, re.IGNORECASE)

    # Capture product name, using an example like "The Classic Potato Chips"
    product_name_match = re.search(r'(\bClassic Potato Chips\b|\bPotato Chips\b)', text, re.IGNORECASE)

    # Determine the correct pack size
    pack_size = "Pack size not found"
    if pack_size_matches:
        # Prefer ounces if both grams and ounces are present
        oz_sizes = [match for match in pack_size_matches if match[1].lower() == 'oz']
        g_sizes = [match for match in pack_size_matches if match[1].lower() == 'g']
        
        if oz_sizes:
            pack_size = f"{oz_sizes[0][0]} OZ"  # Take the first ounce found
        elif g_sizes:
            pack_size = f"{g_sizes[0][0]} g"  # Take the first gram found

    brand_name = brand_name_match.group(1) if brand_name_match else "Brand name not found"
    product_name = product_name_match.group(1) if product_name_match else "Product name not found"

    return {
        "Pack Size": pack_size,
        "Brand Name": brand_name,
        "Product Name": product_name
    }

# EasyOCR extraction
def extract_text_easyocr(image_path):
    reader = easyocr.Reader(['en'])
    extracted_texts = reader.readtext(image_path, detail=0)
    return ' '.join(extracted_texts)

# Hybrid OCR combining Tesseract and EasyOCR
def hybrid_ocr(image, image_path):
    # Tesseract OCR
    tesseract_text = multiple_pass_ocr(image)

    # EasyOCR
    easyocr_text = extract_text_easyocr(image_path)

    # Combine results
    combined_text = ' '.join([tesseract_text, easyocr_text])
    return combined_text

if __name__ == "__main__":
    image_path = 'Lays.jpeg'  # Update this with the path to your image

    # Read the image
    image = cv2.imread(image_path)
    image.show()
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Preprocess the image (adjust contrast, deskew, etc.)
    contrast_image = adjust_contrast(gray_image)
    deskewed_image = deskew_image(contrast_image)

    # Perform hybrid OCR
    combined_text = hybrid_ocr(deskewed_image, image_path)
    # Extract specific details like pack size, brand name, product name using regex
    details = extract_details_using_regex(combined_text)
    
    # Print only the extracted essential details
    print("--- Extracted Details ---")
    print(f"Brand Name: {details['Brand Name']}")
    print(f"Pack Size: {details['Pack Size']}")
    print(f"Product Name: {details['Product Name']}")


[2024-10-11 14:20:50,450] [ WARNING] easyocr.py:80 - Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


--- Extracted Details ---
Brand Name: lays
Pack Size: 10 OZ
Product Name: Classic Potato Chips


In [21]:
import cv2
import pytesseract
import easyocr
import numpy as np
import re

# Manually define the known brands
known_brands = [
    "Lays", "5 Star", "Kitkat", "All Out", "Odonil", "Aashirwaad", 
    "Amul", "Beardo", "Bisleri", "Boost", "Bournvita", "Bru", 
    "Fortune", "Dettol", "Harpic", "Horlicks", "Parle-G", 
    "Kinley", "KrackJack", "Limea", "Maggie", "Yippee", 
    "Marie Gold", "Mirinda", "Munch", "Nestle", "Rin", 
    "Santoor", "Vim", "Sprite", "Stayfree", "Whisper", 
    "Thumbs-Up", "Wildstone"
]

# Image Preprocessing Functions
def adjust_contrast(image):
    return cv2.equalizeHist(image)

def denoise_image(image):
    return cv2.fastNlMeansDenoising(image, None, 30, 7, 21)

def sharpen_image(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def adaptive_threshold(image):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

def deskew_image(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Multiple passes of OCR with different preprocessing techniques
def multiple_pass_ocr(image):
    processed_images = [sharpen_image(image), adaptive_threshold(image), denoise_image(image)]
    all_extracted_text = []
    
    for img in processed_images:
        extracted_text = pytesseract.image_to_string(img, config=r'--oem 3 --psm 6')
        all_extracted_text.append(extracted_text)
    
    return ' '.join(all_extracted_text)

# Regex to extract specific details like brand, pack size, product name, etc.
def extract_details_using_regex(text, known_brands):
    text = text.lower()
    
    pack_size_matches = re.findall(r'(\d+\.?\d*)\s*(oz|g|kg|ml|l)', text)

    brand_name = "Brand name not found"
    for brand in known_brands:
        if brand.lower() in text:
            brand_name = brand  # Keep the original case
            break

    product_name_matches = re.findall(r'(?<=\b' + re.escape(brand_name.lower()) + r'\b.*?)([A-Za-z\s]+)(?=\s+net\s+wt:)', text)
    product_name = product_name_matches[0].strip() if product_name_matches else "Product name not found"

    pack_size = "Pack size not found"
    if pack_size_matches:
        pack_size = f"{pack_size_matches[0][0]} {pack_size_matches[0][1].upper()}"

    return {
        "Pack Size": pack_size,
        "Brand Name": brand_name,
        "Product Name": product_name
    }

# EasyOCR extraction
def extract_text_easyocr(image_path):
    reader = easyocr.Reader(['en'])
    extracted_texts = reader.readtext(image_path, detail=0)
    return ' '.join(extracted_texts)

# Hybrid OCR combining Tesseract and EasyOCR
def hybrid_ocr(image, image_path):
    tesseract_text = multiple_pass_ocr(image)
    easyocr_text = extract_text_easyocr(image_path)
    combined_text = ' '.join([tesseract_text, easyocr_text])
    return combined_text

if __name__ == "__main__":
    image_path = 'Laya.jpeg'  # Update this with the path to your image

    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast_image = adjust_contrast(gray_image)
    deskewed_image = deskew_image(contrast_image)

    combined_text = hybrid_ocr(deskewed_image, image_path)
    print("Combined OCR Text:", combined_text)

    details = extract_details_using_regex(combined_text, known_brands)

    print("--- Extracted Details ---")
    print(f"Brand Name: {details['Brand Name']}")
    print(f"Pack Size: {details['Pack Size']}")
    print(f"Product Name: {details['Product Name']}")


[2024-10-11 14:24:46,814] [ WARNING] easyocr.py:80 - Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Combined OCR Text: ee k Oa i
| ie fe ) ” | o wy ] .
 . on _ oes = a af Sg oe oe as
1 . . a - a st —, — J See ° = ~ i] ]
. NES ye iene VG oy TS TT ee ine - a yt
© HA ie NER cae gt pd pale bi aT — re an ae
Me Ae Sra A ais oe pn) At | 8 Ry ae \ ys
| EB rN Aes ee OOK Wl
; 4 Gy ee ~ — ————_— See si pi 1]
: VE, “ een) eee — a a one: yi ————_—
SSN eT 8 2 8 ed |
} i , i ul i He Ai “e eo rie |
i TLL, Ws ae AUP pl de
a, = Pa 4, Dh aye, bo ° 5 0 a ° ee w] es
4 ey Lee san Tre Tha, a ooo at wt Elst 5
We ea M Cen SES Lge - - aa Zz a Bi
SE a Me) SG, AR a ior ae
 A k ga ‘4
 lays BRAN The Classic Potato Chips Net Wt: 10 OZ. (283.5 g)
--- Extracted Details ---
Brand Name: Brand name not found
Pack Size: 10 OZ
Product Name: ee k Oa i
